In [2]:
import pandas as pd

# Components

### Incidence by region and age groups

In [3]:
incidence = pd.read_csv("incidence.csv")

### Incidence by region and stage

In [4]:
stage_incidence = pd.read_csv("stage_date.csv")

#### F470100

In [5]:
f47_0100_df = pd.read_csv("F470100.csv")
f47_0100_df['category'] = f47_0100_df.category.str.replace(' ', '')
aggregated_f47_0100_df = f47_0100_df[f47_0100_df['category'] == "ВСЬОГО"]

#### F471700 - equipment

In [6]:
f_47_1700_df = pd.read_csv('F471700.csv')

categories_variables = {
    "nx_ray": "Чис.закл.,які мають:рентг.від.(каб)",
    "nflurography": "флюорографiчнi вiддiлення (кабiнети)",
    "nradiology": "радiологiчнi (променевої терапiї)",
    "nradlab": "лабораторії радіоізотопної діагност.",
    "nсt": "комп'ютерної томографії",
    "ncardiogram": "каб.електрокардіог.та функц.діагност.",
    "ndiaglab": "клініко-діагностичні лабораторії",
    "nbacter": "бактеріологічні лабораторії",
    "nbiochem": "біохімічні лабораторії",
    "ncyto": "цитологічні лабораторії",
    "nimun": "імунологічні лабораторії",
    "nphysic": "фізіотерапевтичні відділення (каб.)",
    "nendoscop": "ендоскопічні відділення (кабінети)",
    "nultrasound": "ультразвукової діагностики",
    "ndialysis": "відділення нефрології та діалізу"
}

equipment_dfs = []
for column_name, category in categories_variables.items():
    df = f_47_1700_df[f_47_1700_df['category'] == category][['year', 'region', 'total']]
    df = df.rename(columns={"total": column_name})
    if len(df) == 0:
        raise Exception("inconsistent")
    equipment_dfs.append(df)

equipment_df = equipment_dfs[0]
for df in equipment_dfs[1:]:
    equipment_df = pd.merge(equipment_df, df, on=["year", "region"], how="outer")

### Air pollution

In [7]:
air_pollution_df = pd.read_csv("air_polution.csv")
air_pollution_df = air_pollution_df.rename(columns={'Region': 'region'})

air_pollution_df = air_pollution_df[air_pollution_df['year'] >= 2008]

### Water pollution

In [8]:
water_pollution = pd.read_csv("merged_dumps.csv")
water_pollution = water_pollution.rename(columns={'Region': 'region'})

water_pollution = water_pollution[water_pollution['year'] >= 2008]

### GDP

In [9]:
gdp = pd.read_csv("gdp.csv")
gdp = gdp.rename(columns={"Region": "region"})

gdp = gdp[gdp['year'] >= 2008]

### CPI

In [10]:
cpi = pd.read_csv("inflation.csv", sep=r"\s+")
cpi.cpi = cpi.cpi.map(lambda x: float(x.replace(',', '.')))

cpi = cpi[cpi['year'] >= 2008]

### Population

In [15]:
population = pd.read_csv("population.csv")
population = population.rename(columns={"Територіальний розріз": "region", "Період": "year", "Значення cпостереження": "population"})

population.head()

,region,year,population
0,Автономна Республіка Крим,1989,2063600.0
1,Автономна Республіка Крим,1990,2102400.0
2,Автономна Республіка Крим,1991,2146500.0
3,Автономна Республіка Крим,1992,2184500.0
4,Автономна Республіка Крим,1993,2222700.0


# Datasets

### Age Group - dataset

In [16]:
age_group_incidence = pd.merge(incidence, aggregated_f47_0100_df, on=['year', 'region'], how='outer')
age_group_incidence = pd.merge(age_group_incidence, equipment_df, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, gdp, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, air_pollution_df, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, water_pollution, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, cpi, on="year", how="outer")
age_group_incidence = pd.merge(age_group_incidence, population, on=['year', 'region'], how='outer')

age_group_incidence.to_csv("age_group_incidence.csv")

### Stage dataset

In [18]:
stage_dataset = pd.merge(stage_incidence, aggregated_f47_0100_df, on=['year', 'region'], how='outer')
stage_dataset = pd.merge(stage_dataset, equipment_df, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, gdp, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, air_pollution_df, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, water_pollution, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, cpi, on="year", how="outer")
stage_dataset = pd.merge(stage_dataset, population, on=['year', 'region'], how='outer')

stage_dataset.to_csv("stage_incidence.csv")